In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

In [11]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from tqdm import tqdm
import food.custom_pandas as cpd

from food.qdrant import *


project_name = "food"
table = 'food_categories'
collection_name = 'food_categories'
dim = 768
limit = 100000

client = dev_client



# query = f"""select f.id,f.description, im.*
#         FROM      {project_name}.{table}    f
#         LEFT JOIN {project_name}.indexed      i  ON  (i.id =             f.id)
#         LEFT JOIN {project_name}.image_table  im  ON (im.food_id =       f.id)
        
#         WHERE clip    is not null and 
#               i .indexed is null
#         """

query = f"""select *
        FROM      {project_name}.{table}    f
        WHERE f.clip    is not null"""



In [12]:


for df in tqdm(cpd.read_sql_query(query, engine, chunksize=limit), desc="qdrant_update {project_name}"):

    ids = df['id'].tolist()
    clip = df['clip']
    clip = np.array([np.array(c) for c in clip.values])

    df = df.fillna('0')


    payload = df.drop(columns = ['id','clip']).to_dict('records')

    client.upload_collection(
        collection_name=collection_name,
        vectors=clip,
        payload=payload,
        ids=ids,
        parallel=4
    )





qdrant_update {project_name}: 0it [00:00, ?it/s]2022-06-21 17:55:23.267 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 0 PID: 205641
2022-06-21 17:55:23.270 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 1 PID: 205642
2022-06-21 17:55:23.273 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 2 PID: 205645
2022-06-21 17:55:23.276 | INFO     | qdrant_client.parallel_processor:_worker:45 - Reader worker: 3 PID: 205648
2022-06-21 17:55:23.290 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 3 finished

1it [00:00,  7.51it/s]2022-06-21 17:55:23.390 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 2 finished2022-06-21 17:55:23.442 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 1 finished
2022-06-21 17:55:23.473 | INFO     | qdrant_client.parallel_processor:_worker:77 - Reader worker 0 finished
3it [00:00, 13.81it/s]
qdrant_update {project_name